In [109]:
import tensorflow as tf

x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y + 2
print(f)

Tensor("add_1:0", shape=(), dtype=int32)


In [110]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)

sess.close()

42


In [111]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    

In [112]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()
    print(result)

42


In [113]:
x1 = tf.Variable(1, name='x1')
x1.graph is tf.get_default_graph()

True

In [114]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
x2.graph is graph

True

In [115]:
x2.graph is tf.get_default_graph()

False

In [116]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x + 3

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
8


In [117]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
8


In [118]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()
    print(theta_value)

[[-3.7185181e+01]
 [ 4.3633747e-01]
 [ 9.3952334e-03]
 [-1.0711310e-01]
 [ 6.4479220e-01]
 [-4.0338000e-06]
 [-3.7813708e-03]
 [-4.2348403e-01]
 [-4.3721911e-01]]


In [119]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing.data, housing.target.reshape(-1, 1))

print(np.r_[lin_reg.intercept_.reshape(-1, 1), lin_reg.coef_.T])

[[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]


In [120]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

Epoch 0 MSE = 4.277314


Epoch 100 MSE = 0.74956316


Epoch 200 MSE = 0.636263
Epoch 300 MSE = 0.60327065


Epoch 400 MSE = 0.58180135


Epoch 500 MSE = 0.5663558
Epoch 600 MSE = 0.55512726
Epoch 700 MSE = 0.5469475


Epoch 800 MSE = 0.5409799


Epoch 900 MSE = 0.53661937


In [121]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name="predictions")

error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate*gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()

Epoch 0 MSE = 4.277314


Epoch 100 MSE = 0.74956316


Epoch 200 MSE = 0.636263
Epoch 300 MSE = 0.60327065


Epoch 400 MSE = 0.58180135


Epoch 500 MSE = 0.5663558
Epoch 600 MSE = 0.55512726
Epoch 700 MSE = 0.5469475


Epoch 800 MSE = 0.5409799


Epoch 900 MSE = 0.53661937


In [122]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name="predictions")

error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = tf.gradients(mse, [theta])[0]
training_op = tf.assign(theta, theta - learning_rate*gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()

Epoch 0 MSE = 13.721757
Epoch 100 MSE = 0.93960243
Epoch 200 MSE = 0.6379984
Epoch 300 MSE = 0.6057542
Epoch 400 MSE = 0.58748


Epoch 500 MSE = 0.57380444
Epoch 600 MSE = 0.5633003
Epoch 700 MSE = 0.55517715
Epoch 800 MSE = 0.54886055
Epoch 900 MSE = 0.5439241


In [123]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name="predictions")

error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval(), "theta =", theta.eval())
        sess.run(training_op)
    best_theta = theta.eval()
    print(best_theta)


Epoch 0 MSE = 10.1499405 theta = [[-0.22644806]
 [ 0.12958312]
 [ 0.1116209 ]
 [-0.94798684]
 [-0.7623155 ]
 [-0.13656998]
 [ 0.29201913]
 [-0.9842143 ]
 [-0.10955954]]
Epoch 100 MSE = 0.65500075 theta = [[ 1.7641952 ]
 [ 0.6997564 ]
 [ 0.09192935]
 [-0.04567717]
 [ 0.03874151]
 [-0.06278736]
 [ 0.01182006]
 [-0.6236359 ]
 [-0.5628193 ]]
Epoch 200 MSE = 0.5439741 theta = [[ 2.0281937 ]
 [ 0.7605302 ]
 [ 0.13719639]
 [-0.0597461 ]
 [ 0.08899281]
 [-0.01046662]
 [-0.03153504]
 [-0.6613273 ]
 [-0.61861366]]
Epoch 300 MSE = 0.53687453 theta = [[ 2.063205  ]
 [ 0.7780705 ]
 [ 0.14720964]
 [-0.09071266]
 [ 0.12395354]
 [ 0.00285083]
 [-0.03911394]
 [-0.6955723 ]
 [-0.6560982 ]]


Epoch 400 MSE = 0.5333441 theta = [[ 2.0678484 ]
 [ 0.78799534]
 [ 0.14655365]
 [-0.11847405]
 [ 0.1532116 ]
 [ 0.00516609]
 [-0.04046281]
 [-0.72427255]
 [-0.6866754 ]]
Epoch 500 MSE = 0.53084534 theta = [[ 2.0684638 ]
 [ 0.7954416 ]
 [ 0.14345849]
 [-0.14225295]
 [ 0.17773606]
 [ 0.00466557]
 [-0.04061063]
 [-0.7486821 ]
 [-0.712494  ]]
Epoch 600 MSE = 0.5290423 theta = [[ 2.0685446 ]
 [ 0.8014948 ]
 [ 0.14013728]
 [-0.16239178]
 [ 0.1983696 ]
 [ 0.00357186]
 [-0.04051659]
 [-0.76955396]
 [-0.7345105 ]]
Epoch 700 MSE = 0.5277393 theta = [[ 2.0685523 ]
 [ 0.8065176 ]
 [ 0.13712236]
 [-0.1793819 ]
 [ 0.21575779]
 [ 0.00246129]
 [-0.04038284]
 [-0.7874375 ]
 [-0.7533464 ]]
Epoch 800 MSE = 0.52679724 theta = [[ 2.0685523e+00]
 [ 8.1070524e-01]
 [ 1.3450135e-01]
 [-1.9369133e-01]
 [ 2.3041455e-01]
 [ 1.4670951e-03]
 [-4.0254023e-02]
 [-8.0277658e-01]
 [-7.6948369e-01]]
Epoch 900 MSE = 0.52611625 theta = [[ 2.0685523e+00]
 [ 8.1419802e-01]
 [ 1.3225123e-01]
 [-2.0573024e-01]
 [ 2.4276510e-

[[ 2.0685523e+00]
 [ 8.1710738e-01]
 [ 1.3032651e-01]
 [-2.1584962e-01]
 [ 2.5316647e-01]
 [-1.3017782e-04]
 [-4.0038545e-02]
 [-8.2725388e-01]
 [-7.9519331e-01]]


In [124]:
A = tf.placeholder(tf.float32, shape=(None, 3))
B = A+5

with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A:[[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict={A:[[3, 4, 5]]})

print(B_val_1)
print(B_val_2)

[[6. 7. 8.]]
[[ 8.  9. 10.]]


In [125]:
X = tf.placeholder(tf.float32, shape=(None, n+1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

batch_size = 100
n_batches = int(np.ceil(m/batch_size))
print(n_batches)

207


In [126]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)
    indices = np.random.randint(m, size=batch_size)
    X_batch = scaled_housing_data_plus_bias[indices]
    y_batch = housing.target.reshape(-1, 1)[indices]
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
    best_theta = theta.eval()

In [127]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name="predictions")

error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
            save_path = saver.save(sess, "./my_model.ckpt")
        sess.run(training_op)
    best_theta = theta.eval()
    save_path = saver.save(sess, "./my_model.ckpt")
    print("finish")

Epoch 0 MSE = 8.619016


Epoch 100 MSE = 0.71044266


Epoch 200 MSE = 0.56439334


Epoch 300 MSE = 0.55193293


Epoch 400 MSE = 0.54451656


...

Epoch 600 MSE = 0.5352273


Epoch 700 MSE = 0.532366


Epoch 800 MSE = 0.5302725


Epoch 900 MSE = 0.5287379


finish


In [128]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model.ckpt")
    best_theta_restored = theta.eval()

INFO:tensorflow:Restoring parameters from ./my_model.ckpt


In [129]:
saver = tf.train.import_meta_graph("my_model.ckpt.meta")
theta = tf.get_default_graph().get_tensor_by_name("theta:0")

with tf.Session() as sess:
    saver.restore(sess, "./my_model.ckpt")
    best_theta_restored = theta.eval()

INFO:tensorflow:Restoring parameters from ./my_model.ckpt


In [130]:
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [131]:
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [132]:
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                try:
                    summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                    step = epoch * n_batches + batch_index
                    file_writer.add_summary(summary_str, step)
                except:
                    pass
            try:
                sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            except:
                pass
    best_theta = theta.eval()


In [133]:
file_writer.close()

In [134]:
with tf.name_scope("loss") as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")

In [135]:
print(error.op.name)
print(mse.op.name)

loss/sub
loss/mse


In [136]:
def relu(X):
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random_normal(w_shape), name="weights")
    b = tf.Variable(0.0, name="bias")
    z = tf.add(tf.matmul(X, w), b, name="z")
    return tf.maximum(z, 0., name="relu")

In [137]:
n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

In [138]:
def relu(X, threshold):
    with tf.name_scope("relu"):
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, threshold, name="max")

threshold = tf.Variable(0.0, name="threshold")
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X, threshold) for i in range(5)]
output = tf.add_n(relus, name="output")


In [139]:
#처음 호출할 때 함수의 속성으로 다음과 같이 공유 변수를 지정하는 것 입니다. 

def relu(X):
    with tf.name_scope("relu"):
        if not hasattr(relu, "threshold"):
            relu.threshold = tf.Variable(0.0, name="threshold")
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, relu.threshold, name="max")

In [140]:
with tf.variable_scope("relu"):
    threshold = tf.get_variable("threshold", shape=(),
                                initializer=tf.constant_initializer(0.0))

ValueError: Variable relu/threshold already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\Users\LeeSangHoon\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\ops.py", line 1740, in __init__
    self._traceback = self._graph._extract_stack()  # pylint: disable=protected-access
  File "C:\Users\LeeSangHoon\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\ops.py", line 3414, in create_op
    op_def=op_def)
  File "C:\Users\LeeSangHoon\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 787, in _apply_op_helper
    op_def=op_def)


In [141]:
# 재사용할 수 있는 
with tf.variable_scope("relu", reuse=True):
    threshold = tf.get_variable("threshold")

In [142]:
# 재사용할 수 있는 2
with tf.variable_scope("relu") as scope:
    scope.reuse_variables()
    threshold = tf.get_variable("threshold")

In [143]:


def relu(X):
    with tf.variable_scope("relu", reuse=True):
        threshold = tf.get_variable("threshold")
        w_shape = int(X.get_shape()[1]), 1                          # not shown
        w = tf.Variable(tf.random_normal(w_shape), name="weights")  # not shown
        b = tf.Variable(0.0, name="bias")                           # not shown
        z = tf.add(tf.matmul(X, w), b, name="z")                    # not shown
        return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
with tf.variable_scope("relu"):
    threshold = tf.get_variable("threshold", shape=(),
                                initializer=tf.constant_initializer(0.0))
relus = [relu(X) for relu_index in range(5)]
output = tf.add_n(relus, name="output")

ValueError: Variable relu/threshold already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\Users\LeeSangHoon\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\ops.py", line 1740, in __init__
    self._traceback = self._graph._extract_stack()  # pylint: disable=protected-access
  File "C:\Users\LeeSangHoon\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\ops.py", line 3414, in create_op
    op_def=op_def)
  File "C:\Users\LeeSangHoon\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 787, in _apply_op_helper
    op_def=op_def)


In [144]:
def relu(X):
    with tf.variable_scope("relu"):
        threshold = tf.get_variable("threshold", shape=(), 
                                    initializer=tf.constant_initializer(0))
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = []
for relu_index in range(5):
    with tf.variable_scope("relu", reuse=(relu_index>=1)) as scope:
        relus.append(relu(X))
output = tf.add_n(relus, name='output')

ValueError: Variable relu/relu/threshold already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\Users\LeeSangHoon\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\ops.py", line 1740, in __init__
    self._traceback = self._graph._extract_stack()  # pylint: disable=protected-access
  File "C:\Users\LeeSangHoon\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\ops.py", line 3414, in create_op
    op_def=op_def)
  File "C:\Users\LeeSangHoon\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 787, in _apply_op_helper
    op_def=op_def)


In [145]:
def relu(X):
    threshold = tf.get_variable("threshold", shape=(),
                                initializer=tf.constant_initializer(0.0))
    w_shape = (int(X.get_shape()[1]), 1)                        # not shown in the book
    w = tf.Variable(tf.random_normal(w_shape), name="weights")  # not shown
    b = tf.Variable(0.0, name="bias")                           # not shown
    z = tf.add(tf.matmul(X, w), b, name="z")                    # not shown
    return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = []
for relu_index in range(5):
    with tf.variable_scope("relu", reuse=(relu_index >= 1)) as scope:
        relus.append(relu(X))
output = tf.add_n(relus, name="output")

ValueError: Variable relu/threshold already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\Users\LeeSangHoon\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\ops.py", line 1740, in __init__
    self._traceback = self._graph._extract_stack()  # pylint: disable=protected-access
  File "C:\Users\LeeSangHoon\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\ops.py", line 3414, in create_op
    op_def=op_def)
  File "C:\Users\LeeSangHoon\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 787, in _apply_op_helper
    op_def=op_def)


In [148]:
def relu(X):
    threshold = tf.get_variable("threshold", shape=(),
                                initializer=tf.constant_initializer(0.0))
    w_shape = (int(X.get_shape()[1]), 1)                        # not shown in the book
    w = tf.Variable(tf.random_normal(w_shape), name="weights")  # not shown
    b = tf.Variable(0.0, name="bias")                           # not shown
    z = tf.add(tf.matmul(X, w), b, name="z")                    # not shown
    return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = []
for relu_index in range(5):
    with tf.variable_scope("relu", reuse=True) as scope:
        relus.append(relu(X))
output = tf.add_n(relus, name="output")

In [147]:
print(n_features)

3
